In [ ]:
import numpy as np
import itertools
from qiskit.quantum_info import SparsePauliOp
import qiskit
import random
import qiskit_algorithms

In [ ]:
def construct_hamiltonian(j1, j2, grid):
  def nearest_neighbor(grid, i, j):
    i, j = i % len(grid[0]), j % len(grid)
    look_at = [[1, 0], [-1, 0], [0, 1], [0, -1]]
    result = []
    for element in look_at:
      dx, dy = element
      result.append([(i + dx) % len(grid[0]), (j + dy) % len(grid)])
    return result

  def next_nearest_neighbor(grid, i, j):
    look_at = [[1, 1], [1, -1], [-1, 1], [-1, -1]]
    result = []
    for element in look_at:
      dx, dy = element
      result.append([(i + dx) % len(grid[0]), (j + dy) % len(grid)])
    return result

  def generate_dot_product(grid, term, idxA, idxB):
    operation_template = ['I' for element in range(len(grid[0]) * len(grid))]
    dot_product = SparsePauliOp(('I' * len(grid[0]) * len(grid)), coeffs=[0])
    for direction in ['X', 'Y', 'Z']:
      operation = operation_template
      operation[idxA], operation[idxB] = direction, direction
      dot_product += SparsePauliOp("".join(operation), coeffs=[term])
    return dot_product


  hamilonian = SparsePauliOp(('I' * len(grid[0]) * len(grid)), coeffs=[0])
  for i in range(len(grid[0])):
    for j in range(len(grid)):
      n_neighbors = nearest_neighbor(grid, i, j)
      nn_neighbors = next_nearest_neighbor(grid, i, j)

      for neighbor in n_neighbors:
        idxA = (j * len(grid)) + i
        idxB = (neighbor[1] * len(grid)) + neighbor[0]
        hamilonian += generate_dot_product(grid, j1, idxA, idxB)
      for neighbor in nn_neighbors:
        idxA = (j * len(grid)) + i
        idxB = (neighbor[1] * len(grid)) + neighbor[0]
        hamilonian += generate_dot_product(grid, j2, idxA, idxB)

  return hamilonian.simplify()

In [ ]:
total = 0
import qiskit
def create_excitation_pool(hamiltonian, delta_t=0.01):
    """
    Creates an excitation pool from a Hamiltonian by Trotterizing the evolution
    of each individual term.

    Args:
    - hamiltonian (SummedOp or PauliSumOp): The Hamiltonian to decompose.
    - delta_t (float): The time step for the Trotter evolution.

    Returns:
    - List[PauliSumOp]: A list of Trotterized evolution operators for each term.
    """
    global total

    # Initialize the excitation pool
    excitation_pool = []

    example_pool = []
    num_qubits = len(hamiltonian.paulis[0])
    for i in range(num_qubits):
        example = list("I" * num_qubits)
        example[i] = "X"
        example_pool.append("".join(example))
    for i in range(num_qubits):
        example = list("I" * num_qubits)
        example[i] = "Y"
        example_pool.append("".join(example))
    for i in range(num_qubits):
        example = list("I" * num_qubits)
        example[i] = "Z"
        example_pool.append("".join(example))
    for k1 in range(1):
        for k2 in range(1):
            for i in range(num_qubits):
                for j in range(i+1, num_qubits):
                    example = list("I" * num_qubits)
                    if(k1 == 0):
                        example[i] = "X"
                    else:
                        if(k1 == 1):
                            example[i] = "Y"
                        else:
                            example[i] = "Z"
                    if(k2 == 0):
                        example[j] = "X"
                    else:
                        if(k2 == 1):
                            example[j] = "Y"
                        else:
                            example[j] = "Z"
                    example_pool.append("".join(example))

    example_pool = list(set(example_pool))
    i = 0
    for term in example_pool:
        total += 1
        # Create the evolution operator for the term over delta_t
        actual_term = SparsePauliOp(str(term))
        parameter_prefix='phi' + str(i)+',total' + str(total)
        phi = Parameter(parameter_prefix)
        i += 1
        excitation_pool.append(qiskit.circuit.library.PauliEvolutionGate(actual_term, time=phi))

    return excitation_pool

In [ ]:
import random
from scipy.optimize import minimize
from qiskit.circuit.library import RealAmplitudes
import nevergrad as ng
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import ParameterVector, Parameter
from qiskit.circuit.library import RealAmplitudes
import qiskit
from qiskit.quantum_info import Pauli
from scipy.optimize import dual_annealing

def calculate_energy(angles, coeffs, paulis):
    num_qubits = len(angles) // 2
    energy = 0

    for coeff, pauli_string in zip(coeffs, paulis):
        term_energy = coeff
        for qubit_idx in range(num_qubits):
            theta = angles[2 * qubit_idx]
            phi = angles[2 * qubit_idx + 1]
            contribution = 1
            if 'X' in pauli_string[qubit_idx]:
                contribution *= np.sin(theta) * np.cos(phi)
            elif 'Y' in pauli_string[qubit_idx]:
                contribution *= np.sin(theta) * np.sin(phi)
            elif 'Z' in pauli_string[qubit_idx]:
                contribution *= np.cos(theta)
            term_energy *= contribution
        energy += term_energy
    return energy

def classical_ising_optimization(hamiltonian, num_starts=4):
    coeffs = np.array(hamiltonian.coeffs)
    paulis = [pauli.to_label() for pauli in hamiltonian.paulis]  # Assuming Pauli objects
    num_qubits = hamiltonian.num_qubits

    best_energy = float('inf')
    best_angles = None
    collected_angles = []

    # Bounds for theta [0, pi] and phi [0, 2*pi] for each qubit
    bounds = [(0, np.pi) if i % 2 == 0 else (0, 2 * np.pi) for i in range(2 * num_qubits)]

    for _ in range(num_starts):
        initial_guess = np.random.rand(2 * num_qubits) * np.pi  # Random starting points within bounds

        # Perform optimization
        result = dual_annealing(lambda angles: calculate_energy(angles, coeffs, paulis),
                                bounds=bounds, maxiter=3)

        # Collecting results
        if result.fun < best_energy:
            best_energy = result.fun
            best_angles = result.x
        collected_angles.append(result.x)
        print(result.x)

        print("Initating new optimization...")

    # Averaging angles of the best solutions
    avg_best_angles = np.mean(collected_angles, axis=0)

    return avg_best_angles if best_angles is None else best_angles

In [ ]:
from qiskit.circuit import QuantumCircuit, Parameter

In [ ]:
import numpy as np
from qiskit.primitives import Estimator
from qiskit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.compiler import transpile


from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator, Options


service = QiskitRuntimeService(channel="ibm_quantum", instance="rpi-rensselaer/research/rhone")
backend = service.backend("ibm_rensselaer")

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator, Options
from qiskit.circuit.library import RealAmplitudes
import nevergrad as ng

options = Options()
options.execution.shots = 8127
options.resilience_level = 2
options.resilience.noise_factors = [1.0, 1.25, 1.5, 2, 2.25, 2.5]
options.resilience.extrapolator = "QuadraticExtrapolator"

from qiskit.transpiler import PassManager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes import (
    ALAPScheduleAnalysis,
    PadDynamicalDecoupling,
    ConstrainedReschedule,
)
from qiskit.circuit.library import XGate

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
pm.scheduling = PassManager(
    [
        ALAPScheduleAnalysis(target=target),
        ConstrainedReschedule(target.acquire_alignment, target.pulse_alignment),
        PadDynamicalDecoupling(
            target=target, dd_sequence=[XGate(), XGate()], pulse_alignment=target.pulse_alignment
        ),
    ]
)

import random
import copy
from scipy.optimize import minimize
from qiskit.circuit.library import RealAmplitudes

idx = 1
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    global idx
    try:
        energy = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    except:
        return 1e20
    print("Iteration: ", idx, " Energy: ", energy)
    idx += 1
    return energy


ansatz_global, hamiltonian_global, estimator_global = None, None, None

def make_circuit_initalization(initial_state, qubits):
  qc = QuantumCircuit(qubits)
  for i in range(qubits):
    theta = initial_state[2 * i]
    phi = initial_state[2 * i + 1]
    qc.u(theta, phi, np.random.random(), i)  # U gate is a general single-qubit rotation
  return qc

import random
import copy
import nevergrad as ng
from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit_algorithms.optimizers import SPSA

ansatz = None
averages = []
average_gradient = []
total_losses = []
total_averages = []
comparison_value = [10, 25, 50, 100, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100]
sampled_grads = []
historic_gradients = {}
hamiltonian_temp = None

def run_vqe(hamiltonian, x0=None):
  global ansatz_global
  global hamiltonian_global
  global estimator_global
  global idx
  ansatz = QuantumCircuit(16)
  for k in range(16):
    if(k % 2 == 1):
      ansatz.x(k)
  ansatz = ansatz.compose(TwoLocal(16, 'rx', 'cz', reps=3, entanglement='linear', parameter_prefix='k'))

  bounds = [(0, 2 * np.pi)] * ansatz.num_parameters
  optimizer = NFT(maxiter=200)

  if(type(x0) == type(None)):
     initial_parameters = np.random.random(ansatz.num_parameters) * np.pi * 2
  else:
     initial_parameters = x0
  ansatz_ibm = pm.run(ansatz)
  ansatz_global = ansatz_ibm
  num_params = ansatz.num_parameters
  hamiltonian_ibm = hamiltonian.apply_layout(ansatz_ibm.layout)
  hamiltonian_global = hamiltonian_ibm

  def run_iteration(x0):
    global ansatz_global
    global hamiltonian_global
    global estimator_global

    print(x0)

    cost = cost_func(x0, ansatz_global, hamiltonian_global, estimator_global)
    if(cost == 1e20):
        with Session(backend=backend) as session:
            estimator_global = Estimator(options=options)
            return cost_func(x0, ansatz_global, hamiltonian_global, estimator_global)
    else:
        return cost

  with Session(backend=backend) as session:
    estimator_global = Estimator(options=options)

  res = optimizer.minimize(run_iteration, initial_parameters, bounds=bounds)
  return ansatz, res.x

In [ ]:
list_values = [0.59703065, 5.3916492, 6.22240729, 3.29547571, 0.89171211, 3.7001472,
               7.634675, 4.54657443, 3.0738599, 4.09461365, 1.63000893, 6.19452792,
               3.44579607, 3.00375139, 1.35146942, 6.16654194, 6.21491884, -0.11077545,
               4.89246253, 2.68011777, 2.63952135, 1.43023934, 4.90760085, 0.38722885,
               2.14390662, 1.12834429, 4.61204931, 0.13774911, 4.37556794, 2.16550791,
               5.18104958, 3.20607637, -0.30116758, 2.75948081, 0.06683103, 3.07829204,
               1.08870253, -0.19773182, 2.36507648, 3.22719339, 0.05412471, 2.15543781,
               2.23548472, 3.19946806, 4.17465195, 5.80438401, 0.54738063, -0.45223634,
               5.61228832, 4.08091898, 3.48022833, 5.22394854, 5.29200529, 6.65191819,
               3.80878882, 0.81096679, 5.18468394, 1.10466176, 6.18111814, 0.35730182,
               0.67985873, 1.01704788, 5.38947754, 3.51349223]
ansatz_ibm, x0 = run_vqe(construct_hamiltonian(1.0, 0.8, [[1/2 for i in range(4)] for j in range(4)]), np.array(list_values))

[ 0.59703065  5.3916492   6.22240729  3.29547571  0.89171211  3.7001472
  7.634675    4.54657443  3.0738599   4.09461365  1.63000893  6.19452792
  3.44579607  3.00375139  1.35146942  6.16654194  6.21491884 -0.11077545
  4.89246253  2.68011777  2.63952135  1.43023934  4.90760085  0.38722885
  2.14390662  1.12834429  4.61204931  0.13774911  4.37556794  2.16550791
  5.18104958  3.20607637 -0.30116758  2.75948081  0.06683103  3.07829204
  1.08870253 -0.19773182  2.36507648  3.22719339  0.05412471  2.15543781
  2.23548472  3.19946806  4.17465195  5.80438401  0.54738063 -0.45223634
  5.61228832  4.08091898  3.48022833  5.22394854  5.29200529  6.65191819
  3.80878882  0.81096679  5.18468394  1.10466176  6.18111814  0.35730182
  0.67985873  1.01704788  5.38947754  3.51349223]


base_runtime_job._start_websocket_client:WARNING:2024-04-11 10:30:31,834: An error occurred while streaming results from the server for job crbz72c78n6g008dys50:
Traceback (most recent call last):
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\base_runtime_job.py", line 311, in _start_websocket_client
    self._ws_client.job_results()
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\api\clients\runtime_ws.py", line 70, in job_results
    self.stream(url=url, retries=max_retries, backoff_factor=backoff_factor)
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\api\clients\base_websocket_client.py", line 222, in stream
    raise WebsocketError(error_message)
qiskit_ibm_runtime.api.exceptions.WebsocketError: 'Max retries exceeded: Failed to establish a websocket connection.'



Iteration:  1  Energy:  -39.66876581318235
[ 2.16782698  5.3916492   6.22240729  3.29547571  0.89171211  3.7001472
  7.634675    4.54657443  3.0738599   4.09461365  1.63000893  6.19452792
  3.44579607  3.00375139  1.35146942  6.16654194  6.21491884 -0.11077545
  4.89246253  2.68011777  2.63952135  1.43023934  4.90760085  0.38722885
  2.14390662  1.12834429  4.61204931  0.13774911  4.37556794  2.16550791
  5.18104958  3.20607637 -0.30116758  2.75948081  0.06683103  3.07829204
  1.08870253 -0.19773182  2.36507648  3.22719339  0.05412471  2.15543781
  2.23548472  3.19946806  4.17465195  5.80438401  0.54738063 -0.45223634
  5.61228832  4.08091898  3.48022833  5.22394854  5.29200529  6.65191819
  3.80878882  0.81096679  5.18468394  1.10466176  6.18111814  0.35730182
  0.67985873  1.01704788  5.38947754  3.51349223]


base_runtime_job._start_websocket_client:WARNING:2024-04-11 10:34:02,324: An error occurred while streaming results from the server for job crbz828622j0008m83fg:
Traceback (most recent call last):
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\base_runtime_job.py", line 311, in _start_websocket_client
    self._ws_client.job_results()
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\api\clients\runtime_ws.py", line 70, in job_results
    self.stream(url=url, retries=max_retries, backoff_factor=backoff_factor)
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\api\clients\base_websocket_client.py", line 222, in stream
    raise WebsocketError(error_message)
qiskit_ibm_runtime.api.exceptions.WebsocketError: 'Max retries exceeded: Failed to establish a websocket connection.'



Iteration:  2  Energy:  -52.735116835166316
[-0.97376568  5.3916492   6.22240729  3.29547571  0.89171211  3.7001472
  7.634675    4.54657443  3.0738599   4.09461365  1.63000893  6.19452792
  3.44579607  3.00375139  1.35146942  6.16654194  6.21491884 -0.11077545
  4.89246253  2.68011777  2.63952135  1.43023934  4.90760085  0.38722885
  2.14390662  1.12834429  4.61204931  0.13774911  4.37556794  2.16550791
  5.18104958  3.20607637 -0.30116758  2.75948081  0.06683103  3.07829204
  1.08870253 -0.19773182  2.36507648  3.22719339  0.05412471  2.15543781
  2.23548472  3.19946806  4.17465195  5.80438401  0.54738063 -0.45223634
  5.61228832  4.08091898  3.48022833  5.22394854  5.29200529  6.65191819
  3.80878882  0.81096679  5.18468394  1.10466176  6.18111814  0.35730182
  0.67985873  1.01704788  5.38947754  3.51349223]


...

Iteration:  36  Energy:  -45.614356717939955
[ 2.50739662  4.94088775  6.08502295  3.65144021  0.94730885  3.9005884
  7.46188884  5.08502355  2.87809207  4.61222016  1.06561859  6.53325471
  3.26145576  3.31571661  1.81855752  5.99621044  6.46794231 -1.68157178
  4.89246253  2.68011777  2.63952135  1.43023934  4.90760085  0.38722885
  2.14390662  1.12834429  4.61204931  0.13774911  4.37556794  2.16550791
  5.18104958  3.20607637 -0.30116758  2.75948081  0.06683103  3.07829204
  1.08870253 -0.19773182  2.36507648  3.22719339  0.05412471  2.15543781
  2.23548472  3.19946806  4.17465195  5.80438401  0.54738063 -0.45223634
  5.61228832  4.08091898  3.48022833  5.22394854  5.29200529  6.65191819
  3.80878882  0.81096679  5.18468394  1.10466176  6.18111814  0.35730182
  0.67985873  1.01704788  5.38947754  3.51349223]


base_runtime_job._start_websocket_client:WARNING:2024-04-11 12:28:14,279: An error occurred while streaming results from the server for job crc0xrq622j0008m84gg:
Traceback (most recent call last):
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\base_runtime_job.py", line 311, in _start_websocket_client
    self._ws_client.job_results()
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\api\clients\runtime_ws.py", line 70, in job_results
    self.stream(url=url, retries=max_retries, backoff_factor=backoff_factor)
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\api\clients\base_websocket_client.py", line 222, in stream
    raise WebsocketError(error_message)
qiskit_ibm_runtime.api.exceptions.WebsocketError: 'Max retries exceeded: Failed to establish a websocket connection.'



Iteration:  37  Energy:  -45.67712832212145
[ 2.50739662  4.94088775  6.08502295  3.65144021  0.94730885  3.9005884
  7.46188884  5.08502355  2.87809207  4.61222016  1.06561859  6.53325471
  3.26145576  3.31571661  1.81855752  5.99621044  6.46794231 -0.1126644
  6.46325886  2.68011777  2.63952135  1.43023934  4.90760085  0.38722885
  2.14390662  1.12834429  4.61204931  0.13774911  4.37556794  2.16550791
  5.18104958  3.20607637 -0.30116758  2.75948081  0.06683103  3.07829204
  1.08870253 -0.19773182  2.36507648  3.22719339  0.05412471  2.15543781
  2.23548472  3.19946806  4.17465195  5.80438401  0.54738063 -0.45223634
  5.61228832  4.08091898  3.48022833  5.22394854  5.29200529  6.65191819
  3.80878882  0.81096679  5.18468394  1.10466176  6.18111814  0.35730182
  0.67985873  1.01704788  5.38947754  3.51349223]


base_runtime_job._start_websocket_client:WARNING:2024-04-11 12:31:34,345: An error occurred while streaming results from the server for job crc0z7w622j0008m84hg:
Traceback (most recent call last):
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\base_runtime_job.py", line 311, in _start_websocket_client
    self._ws_client.job_results()
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\api\clients\runtime_ws.py", line 70, in job_results
    self.stream(url=url, retries=max_retries, backoff_factor=backoff_factor)
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\api\clients\base_websocket_client.py", line 222, in stream
    raise WebsocketError(error_message)
qiskit_ibm_runtime.api.exceptions.WebsocketError: 'Max retries exceeded: Failed to establish a websocket connection.'



Iteration:  38  Energy:  -50.074571672152445
[ 2.50739662  4.94088775  6.08502295  3.65144021  0.94730885  3.9005884
  7.46188884  5.08502355  2.87809207  4.61222016  1.06561859  6.53325471
  3.26145576  3.31571661  1.81855752  5.99621044  6.46794231 -0.1126644
  3.3216662   2.68011777  2.63952135  1.43023934  4.90760085  0.38722885
  2.14390662  1.12834429  4.61204931  0.13774911  4.37556794  2.16550791
  5.18104958  3.20607637 -0.30116758  2.75948081  0.06683103  3.07829204
  1.08870253 -0.19773182  2.36507648  3.22719339  0.05412471  2.15543781
  2.23548472  3.19946806  4.17465195  5.80438401  0.54738063 -0.45223634
  5.61228832  4.08091898  3.48022833  5.22394854  5.29200529  6.65191819
  3.80878882  0.81096679  5.18468394  1.10466176  6.18111814  0.35730182
  0.67985873  1.01704788  5.38947754  3.51349223]


In [ ]:
print(ansatz_ibm)

           »
 q_0: ─────»
      ┌───┐»
 q_1: ┤ X ├»
      └───┘»
 q_2: ─────»
      ┌───┐»
 q_3: ┤ X ├»
      └───┘»
 q_4: ─────»
      ┌───┐»
 q_5: ┤ X ├»
      └───┘»
 q_6: ─────»
      ┌───┐»
 q_7: ┤ X ├»
      └───┘»
 q_8: ─────»
      ┌───┐»
 q_9: ┤ X ├»
      └───┘»
q_10: ─────»
      ┌───┐»
q_11: ┤ X ├»
      └───┘»
q_12: ─────»
      ┌───┐»
q_13: ┤ X ├»
      └───┘»
q_14: ─────»
      ┌───┐»
q_15: ┤ X ├»
      └───┘»
«      ┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
« q_0: ┤0                                                                                                                                                                     

In [ ]:
def neel_order(dim):
    L = dim[0] * dim[1]
    neel_op = SparsePauliOp(('I' * L), coeffs=[0])

    for i in range(L):
        x, y = i % dim[0], i // dim[0]
        sign = (-1) ** (x + y)
        label = ['I'] * L
        label[i] = 'Z'
        neel_op += SparsePauliOp(''.join(label), coeffs=[sign])

    neel_op = neel_op.simplify()
    neel_op /= L  # Normalize the Néel operator

    return (neel_op @ neel_op).simplify()

def dimer_order(dim):
    Lx, Ly = dim
    num_spins = Lx*Ly
    dimer_op = SparsePauliOp(('I' * num_spins), coeffs=[0])
    normalization = 0

    for x in range(0, Lx//2, 2):
        for y in range(Ly):
            i = y * Ly + x
            j = ((x + 1) % Lx) + y * Ly
            sign = (-1)**(x)
            label = ['I'] * num_spins
            label[i] = label[j] = 'X'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Y'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Z'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            normalization += 1

    dimer_op.simplify()

    for x in range(0, Lx//2, 3):
        for y in range(Ly):
            i = y * Ly + x
            j = ((x + 2) % Lx) + y * Ly
            sign = (-1)**(x)
            label = ['I'] * num_spins
            label[i] = label[j] = 'X'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Y'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Z'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            normalization += 1

    dimer_op.simplify()

    for y in range(0, Ly//2, 2):
        for x in range(Lx):
            i = y * Ly + x
            j = ((y + 1) % Ly)*Ly + x
            sign = (-1)**(y)
            label = ['I'] * num_spins
            label[i] = label[j] = 'X'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Y'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Z'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            normalization += 1

    dimer_op.simplify()

    for y in range(0, Ly//2, 3):
        for x in range(Lx):
            i = y * Ly + x
            j = ((y + 2) % Ly)*Ly + x
            sign = (-1)**(y)
            label = ['I'] * num_spins
            label[i] = label[j] = 'X'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Y'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Z'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            normalization += 1

    dimer_op.simplify()
    dimer_op /= normalization
    return (dimer_op @ dimer_op).simplify()

def spin_order(dim):
    L = dim[0] * dim[1]
    spin_corr = SparsePauliOp(('I' * L), coeffs=[0])

    for i in range(L):
        for j in range(i+1, L):
            x1, y1 = i % dim[0], i // dim[0]
            x2, y2 = j % dim[0], j // dim[0]

            if abs(x1 - x2) + abs(y1 - y2) == 1:  # Nearest neighbors
                label = ['I'] * L
                label[i] = 'Z'
                label[j] = 'Z'
                spin_corr += SparsePauliOp(''.join(label), coeffs=[1])

    spin_corr = spin_corr.simplify()
    spin_corr /= L  # Normalize the spin correlation operator

    return spin_corr

def spin_correlation(dim, max_distance=None):
    L = dim[0] * dim[1]
    spin_corr = SparsePauliOp(('I' * L), coeffs=[0])

    for i in range(L):
        for j in range(i+1, L):
            x1, y1 = i % dim[0], i // dim[0]
            x2, y2 = j % dim[0], j // dim[0]

            distance = abs(x1 - x2) + abs(y1 - y2)
            if max_distance is None or distance <= max_distance:
                label = ['I'] * L
                label[i] = 'Z'
                label[j] = 'Z'
                coeff = 1 / (distance ** 2)  # Weight coefficient by inverse square of distance
                spin_corr += SparsePauliOp(''.join(label), coeffs=[coeff])

    spin_corr = spin_corr.simplify()
    spin_corr /= L  # Normalize the spin correlation operator

    return spin_corr

In [ ]:
dim = [4,4]
neel_ham = neel_order(dim)
dimer_ham = dimer_order(dim)
spin_ham = spin_order(dim)
spin_corr_ham = spin_correlation(dim)
neel_ham = neel_ham.apply_layout(ansatz_ibm2.layout)
dimer_ham = dimer_ham.apply_layout(ansatz_ibm2.layout)
spin_ham = spin_ham.apply_layout(ansatz_ibm2.layout)
spin_corr_ham = spin_corr_ham.apply_layout(ansatz_ibm2.layout)

In [ ]:
ansatz_ibm2 = pm.run(ansatz_ibm)
print(ansatz_ibm2)

global phase: 2π
                   ┌──────────────────┐                           »
    ancilla_0 -> 0 ┤ Delay(64736[dt]) ├───────────────────────────»
                   ├──────────────────┤                           »
    ancilla_1 -> 1 ┤ Delay(64736[dt]) ├───────────────────────────»
                   ├──────────────────┤                           »
    ancilla_2 -> 2 ┤ Delay(64736[dt]) ├───────────────────────────»
                   ├──────────────────┤                           »
    ancilla_3 -> 3 ┤ Delay(64736[dt]) ├───────────────────────────»
                   ├──────────────────┤                           »
    ancilla_4 -> 4 ┤ Delay(64736[dt]) ├───────────────────────────»
                   ├──────────────────┤                           »
    ancilla_5 -> 5 ┤ Delay(64736[dt]) ├───────────────────────────»
                   ├──────────────────┤                           »
    ancilla_6 -> 6 ┤ Delay(64736[dt]) ├───────────────────────────»
                   ├───────────

In [ ]:
with Session(backend=backend) as session:
    estimator = Estimator(options=options)
    print(estimator.run([ansatz_ibm2 for i in range(4)], [neel_ham, dimer_ham, spin_ham, spin_corr_ham], parameter_values=[x0 for i in range(4)]).result().values)

base_runtime_job._start_websocket_client:WARNING:2024-04-11 09:27:21,187: An error occurred while streaming results from the server for job crby2w3c90400089h3q0:
Traceback (most recent call last):
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\base_runtime_job.py", line 311, in _start_websocket_client
    self._ws_client.job_results()
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\api\clients\runtime_ws.py", line 70, in job_results
    self.stream(url=url, retries=max_retries, backoff_factor=backoff_factor)
  File "C:\Users\Matthew\.conda\envs\qiskit1\lib\site-packages\qiskit_ibm_runtime\api\clients\base_websocket_client.py", line 222, in stream
    raise WebsocketError(error_message)
qiskit_ibm_runtime.api.exceptions.WebsocketError: 'Max retries exceeded: Failed to establish a websocket connection.'



[ 0.17003917  0.19407657 -0.51984945 -0.43597231]


In [ ]:
idx = 0
for element in callback_dict['cost_history']:
    print("(", idx + 1, ", ", element, ")")
    idx += 1